In [ ]:
# Plot options
import matplotlib.pyplot as plt

# Import required libraries
import numpy as np

# Import karabo libraries
import karabo_bridge as kb
import karabo_data as kd
import xfelmay2019 as xfel


# Run 189 min and max intensity

In [ ]:
path='/gpfs/exfel/exp/SQS/201921/p002430/raw'
run = xfel.runFormat( 189 )
runData189 = kd.RunDirectory(path+run)

In [ ]:
scatImages = np.asarray(runData189.get_array('SQS_DPU_LIC/CAM/YAG_UPSTR:daqOutput', 'data.image.pixels'))

In [ ]:
scatInt = np.sum( scatImages, (1,2) )

In [ ]:
indMax = np.argmax(scatInt)
indMin = np.argmin(scatInt)

In [ ]:
plt.figure(3)
plt.imshow(scatImages[indMax])

In [ ]:
plt.figure(3)
plt.imshow(scatImages[indMin])

# Run 190 min and max intensity

In [ ]:
path='/gpfs/exfel/exp/SQS/201921/p002430/raw'
run = xfel.runFormat( 190 )
runData190 = kd.RunDirectory(path+run)

In [ ]:
scatImages = np.asarray(runData190.get_array('SQS_DPU_LIC/CAM/YAG_UPSTR:daqOutput', 'data.image.pixels'))

In [ ]:
scatInt = np.sum( scatImages, (1,2) )

In [ ]:
indMax = np.argmax(scatInt)
indMin = np.argmin(scatInt)

In [ ]:
plt.figure(3)
plt.imshow(scatImages[indMax])

In [ ]:
plt.figure(3)
plt.imshow(scatImages[indMin])

# Run 191 min and max intensity

In [ ]:
path='/gpfs/exfel/exp/SQS/201921/p002430/raw'
run = xfel.runFormat( 191 )
runData191 = kd.RunDirectory(path+run)

In [ ]:
scatImages191 = np.asarray(runData191.get_array('SQS_DPU_LIC/CAM/YAG_UPSTR:daqOutput', 'data.image.pixels'))

In [ ]:
scatInt191 = np.sum( scatImages191, (1,2) )

In [ ]:
indMax = np.argmax(scatInt191)
indMin = np.argmin(scatInt191)

In [ ]:
scatmax191=scatImages191[indMax]

In [ ]:
plt.figure(3)
plt.imshow(scatImages191[indMax])

In [ ]:
plt.figure(3)
plt.imshow(scatImages[indMin])

# Run 183 min and max intensity

In [ ]:
path='/gpfs/exfel/exp/SQS/201921/p002430/raw'
run = xfel.runFormat( 183 )
runData183 = kd.RunDirectory(path+run)

In [ ]:
scatImages183 = np.asarray(runData183.get_array('SQS_DPU_LIC/CAM/YAG_UPSTR:daqOutput', 'data.image.pixels'))

In [ ]:
scatInt183 = np.sum( scatImages183, (1,2) )

In [ ]:
indMax = np.argmax(scatInt183)
indMin = np.argmin(scatInt183)

In [ ]:
scatmax183=scatImages183[indMax]

In [ ]:
plt.figure(3)
plt.imshow(scatImages183[indMax])

In [ ]:
plt.figure(3)
plt.imshow(scatImages[indMin])

# Max - Min

In [ ]:
plt.figure(3)
plt.imshow( np.log(np.abs(scatmax191/np.sum(scatmax191.flatten()) -   scatmax183/np.sum(scatmax183.flatten())))  )

# Run 230 min and max intensity

In [ ]:
path='/gpfs/exfel/exp/SQS/201921/p002430/raw'
run = xfel.runFormat( 230 )
runData189 = kd.RunDirectory(path+run)

In [ ]:
scatImages = np.asarray(runData189.get_array('SQS_DPU_LIC/CAM/YAG_UPSTR:daqOutput', 'data.image.pixels'))

In [ ]:
scatInt = np.sum( scatImages, (1,2) )

In [ ]:
indMax = np.argmax(scatInt)
indMin = np.argmin(scatInt)

In [ ]:
plt.figure(3)
plt.imshow(scatImages[indMax])

In [ ]:
plt.figure(3)
plt.imshow(scatImages[indMin])

In [ ]:
scatMin = (scatImages[indMin])
scatMax = (scatImages[indMax])

plt.imshow(scatMax/scatMax.flatten().max() - scatMin/scatMin.flatten().max())